# Data Cleaning - Sentiment analysis

## Load Data

In [15]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rdolphin/financial-news-with-ticker-level-sentiment")

print("Path to dataset files:", path)

Path to dataset files: /Users/hippolyteheger/.cache/kagglehub/datasets/rdolphin/financial-news-with-ticker-level-sentiment/versions/1


In [16]:
import os
import pandas as pd

print(os.listdir(path))

['polygon_news_sample.json']


In [17]:
import pandas as pd
import json

file_path = f"{path}/polygon_news_sample.json"

with open(file_path, "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.head()

,article_url,author,description,id,image_url,amp_url,keywords,published_utc,publisher,tickers,insights,title
0,https://www.zacks.com/stock/news/2114673/alleg...,Zacks.com,Allegiant Travel (ALGT) is a fast-moving stock...,db221630f08d9064b6539534cc9957ecd7ba2a626927c7...,https://staticx-tuner.zacks.com/images/default...,None,"[Allegiant Travel, momentum investing, fast-pa...",2023-06-29T12:50:06Z,"{'name': 'Zacks Investment Research', 'homepag...",[ALGT],"[{'ticker': 'ALGT', 'sentiment': 'positive', '...",Allegiant Travel (ALGT) Is Attractively Priced...
1,https://www.zacks.com/stock/news/2085677/appli...,Zacks.com,Applied Industrial Technologies (AIT) reported...,bb7e1725949a7254ae18e8d149c3c19af050c0ac05f18f...,https://staticx-tuner.zacks.com/images/default...,None,"[earnings, revenues, estimates, industrial pro...",2023-04-27T11:55:14Z,"{'name': 'Zacks Investment Research', 'homepag...","[AIT, NPO]","[{'ticker': 'AIT', 'sentiment': 'positive', 's...",Applied Industrial Technologies (AIT) Q3 Earni...
2,https://www.globenewswire.com/news-release/202...,N/A,"Apollo Commercial Real Estate Finance, Inc. (A...",a49c53ef44092946950dfb3f33852c9ef07d7c7dc6c1ea...,https://ml.globenewswire.com/Resource/Download...,None,"[commercial real estate, financing, mortgage l...",2023-03-06T13:30:00Z,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...","[ARI, SAN]","[{'ticker': 'ARI', 'sentiment': 'positive', 's...","Apollo Commercial Real Estate Finance, Inc. Cl..."
3,https://www.globenewswire.com/news-release/202...,N/A,"Maravai LifeSciences, a global provider of lif...",be4f5174307cd0f3309ee931ab4ec4fc2451af056769ca...,https://ml.globenewswire.com/Resource/Download...,None,"[Maravai LifeSciences, investor conferences, f...",2023-11-09T13:15:00Z,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...",[MRVI],"[{'ticker': 'MRVI', 'sentiment': 'positive', '...",Maravai LifeSciences Announces November 2023 I...
4,https://www.zacks.com/stock/news/2069321/dht-h...,Zacks Equity Research,"DHT Holdings, an independent oil tanker compan...",29bea2bb15df75a10fd940c2dc705d21d4c413fb45c17a...,https://staticx-tuner.zacks.com/images/default...,None,"[DHT Holdings, oil tanker, earnings, revenue, ...",2023-03-22T22:00:25Z,"{'name': 'Zacks Investment Research', 'homepag...",[DHT],"[{'ticker': 'DHT', 'sentiment': 'neutral', 'se...",DHT Holdings (DHT) Stock Moves -1.33%: What Yo...


In [18]:
df.columns

Index(['article_url', 'author', 'description', 'id', 'image_url', 'amp_url',
       'keywords', 'published_utc', 'publisher', 'tickers', 'insights',
       'title'],
      dtype='object')

In [19]:
df.dtypes

article_url      object
author           object
description      object
id               object
image_url        object
amp_url          object
keywords         object
published_utc    object
publisher        object
tickers          object
insights         object
title            object
dtype: object

## Data Cleaning

In [20]:
df[['sentiment', 'sentiment_reasoning']] = df['insights'].apply(lambda x: pd.Series({
    'sentiment': x[0]['sentiment'],
    'sentiment_reasoning': x[0]['sentiment_reasoning']
}))

In [21]:
df = df[['description', 'sentiment','sentiment_reasoning',
       'title']]

In [22]:
#df['published_utc'] = pd.to_datetime(df['published_utc']).dt.date
1#df = df.set_index('published_utc')


1

In [ ]:
def extract_sentiments(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts sentiment and sentiment_reasoning from the 'insights' column,
    and returns a DataFrame with only the relevant columns:
    ['description', 'sentiment', 'sentiment_reasoning', 'title'].

    Args:
        df (pd.DataFrame): Original DataFrame with 'insights' column.

    Returns:
        pd.DataFrame: Processed DataFrame with extracted sentiment info.
    """
    # Extract 'sentiment' and 'sentiment_reasoning' from insights
    df[['sentiment', 'sentiment_reasoning']] = df['insights'].apply(
        lambda x: pd.Series({
            'sentiment': x[0]['sentiment'],
            'sentiment_reasoning': x[0]['sentiment_reasoning']
        })
    )

    # Keep only the relevant columns
    df = df[['description', 'sentiment', 'sentiment_reasoning', 'title']]

    return df